In [2]:
import os
from collections import namedtuple

os.chdir("../")

In [3]:
DataIngestionConfig = namedtuple("DataIngestionConfig", ["root_dir" ,"source_URL" , 
                                 "local_data_file", "unzip_dir"])

In [4]:
from deepClassifier.constants import *
from deepClassifier.utils import create_directories, read_yaml

In [5]:
class ConfigurationManager:
    def __init__(self , 
                 config_file_path = CONFIG_FILE_PATH , 
                 param_file_path = PARAMS_FILE_PATH) -> None:
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(param_file_path)
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion 
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(root_dir = config.root_dir
                                                    ,source_URL= config.source_URL , 
                                                    local_data_file= config.local_data_file , 
                                                    unzip_dir = config.unzip_dir)
        return data_ingestion_config

In [6]:
import os 
import urllib.request as request 
from zipfile import ZipFile

In [7]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )

    def _get_updated_list_of_files(self, list_of_files):
        return [f for f in list_of_files if f.endswith(".jpg") and ("Cat" in f or "Dog" in f)]

    def _preprocess(self, zf: ZipFile, f: str, working_dir: str):
        target_filepath = os.path.join(working_dir, f)
        if not os.path.exists(target_filepath):
            zf.extract(f, working_dir)
        
        if os.path.getsize(target_filepath) == 0:
            os.remove(target_filepath)

    def unzip_and_clean(self):
        with ZipFile(file=self.config.local_data_file, mode="r") as zf:
            list_of_files = zf.namelist()
            updated_list_of_files = self._get_updated_list_of_files(list_of_files)
            for f in updated_list_of_files:
                self._preprocess(zf, f, self.config.unzip_dir)

In [7]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzip_and_clean()
except Exception as e:
    raise e

2022-09-22 04:28:06.926 | INFO     | deepClassifier.utils.common:read_yaml:30 - yaml file: configs/config.yaml loaded successfully
2022-09-22 04:28:06.928 | INFO     | deepClassifier.utils.common:read_yaml:30 - yaml file: params.yaml loaded successfully
2022-09-22 04:28:06.929 | INFO     | deepClassifier.utils.common:create_directories:49 - created directory at: artifacts
2022-09-22 04:28:06.929 | INFO     | deepClassifier.utils.common:create_directories:49 - created directory at: artifacts/data_ingestion
